<a href="https://colab.research.google.com/github/ttvhh/CS114.K21/blob/master/%5BCase_Study%5D_Sarcasm_Detection_v1_VI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# **MÔ TẢ BÀI TOÁN**

Con người có bản năng xã hội, tức là chúng ta tương tác với nhau theo những cách tích cực, thân thiện, và bản năng này còn mang một ý nghĩa chính là thao túng, điều khiển, châm chọc bất kì ai đó theo một cách tiêu cực.

***Mỉa mai*** (*sarcasm*) vừa mang một khía cạnh hài hước tích cực mà cũng mang theo đó là những sự khó chịu tiêu cực. Vì vậy mà ***mỉa mai*** đóng một vai trò quan trọng trong tương tác xã hội của người. 

Tuy nhiên, đôi khi rất khó để có thể phát hiện được ai đó đang trêu chọc chúng ta với những lời lẽ mỉa mai hay không. Do đó, ở đây chúng ta sẽ xây dụng một mô hình có thể giúp chúng ta phát hiện những câu nói, lời văn mang tính mỉa mai.

## **CÁC BƯỚC XÂY DỰNG MÔ HÌNH DỰ ĐOÁN**


1.   Thu thập dữ liệu
2.   Xử lý dữ liệu
3.   Lựa chọn model
4.   Học và fine-tuning




###1. THU THẬP DỮ LIỆU

Dựa vào yêu cầu bài toán đặt ra, trước hết ta có được bộ News Headlines Dataset từ trang chuyên về ML/DL như Kaggle, và mục tiêu của chúng ta chính là dự đoán xem một lời văn có mang tính mỉa mai hay không.

Bộ dataset này được thu thập từ hai trang web khác nhau, trong đó lần lượt ***The Onion*** tập trung chủ yếu đăng tin mỉa mai, châm biếm và ***HuffPost*** đăng những bài báo có độ tin cậy, chính xác cao.

Ta có thể lấy bộ dataset này ở đường link phía dưới:
> [News Headlines Dataset For Sarcasm Detection](https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection/kernels)

**Mô tả dataset**

*   ```article_link``` (type: Object): chứa những đường link dẫn tới trang tin tức tương ứng.

*   ```headline``` (type: Object): chứa tiêu đề của những trang tin tức tương ứng.

*   ```is_sarcastic``` (type: int64): chứa 0 (không mỉa mai) và 1 (mỉa mai).



Để có thể thu thập 2000 dữ liệu từ hai trang trên mà không sử dụng bộ dataset có sẵn thì ta phải xây dựng một ***web crawler*** chịu trách nhiệm tự động thu thập đường dẫn, tiêu đề và đánh nhãn 0 (đối với trang ***The Onion***) và 1 (đối với trang ***HuffPost***).

Trước khi thu thập dữ liệu, ta cần cài đặt và import một số thư viện liên quan

In [2]:
!pip install pandas requests bs4

In [3]:
import bs4
import pandas
import requests


####1.1. Scrape dữ liệu
Sau đó, ta sử dụng hàm ```request.get``` để lấy dữ liệu từ trang TheOnion về. Thư viện requests sẽ tạo ra 1 yêu cầu GET cho máy chủ web để tải xuống nội dung HTML của trang web ta đang cần.

In [4]:
url = 'https://www.theonion.com/latest'
def get_page_content(url):
   page = requests.get(url,headers={"Accept-Language":"en-US"})
   return bs4.BeautifulSoup(page.text,"html.parser")
soup = get_page_content(url)


####1.2. Trích xuất dữ liệu thô

Ở mục này, ta sẽ lần lượt trích xuất từng dữ liệu thô của trang như: title, đường link tương ứng, url cho trang kế tiếp

Việc chúng ta muốn lấy trường dữ liệu nào trong số dữ liệu đã được crawl về thì ta chỉ cần click chuột phải vào trang web, chọn ***Inspect element***. Sau đó tìm đến thuộc tính tương ứng.

Code cell bên dưới tạo 2 mảng (titles, article_links) nhằm mục đích lưu trữ từng trường dữ liệu khác nhau. Từ đây, sẽ thực hiện lần lượt từng bước để trích xuất dữ liệu rồi lưu vào mảng kể trên.

In [5]:
titles = []
article_link = []
count = 0
while(count < 1500):
  soup = get_page_content(url)
  #Lấy đường title cho mỗi bài báo
  articles = soup.findAll('h2', class_="sc-759qgu-0 cYlVdn cw4lnv-6 eXwNRE")

  titles += [str(article.text) for article in articles]

  # Lấy đường link tương ứng cho mỗi article
  articles = soup.find_all('a', class_="sc-1out364-0 hMndXN js_link")

  for article in articles:
    h2_element = article.find('h2', class_="sc-759qgu-0 cYlVdn cw4lnv-6 eXwNRE")
    if h2_element:
      article_link.append(article['href'])
  # Lấy đường url cho page tiếp theo
  for article in articles:
    if article.find('button'):
      url = 'https://politics.theonion.com/' + article['href']

  count = len(titles)



####1.3. Trình bày dữ liệu với Pandas

Bước này ta bắt đầu gán nhãn 1 cho toàn bộ dữ liệu thu thập được do ta vừa crawl dữ liệu về từ trang ***TheOnion***

Tiếp theo đó, để biểu diễn dữ liệu một cách trực quan ta chỉ cần sử dụng ```pandas.DataFrame()```

In [6]:
is_sarcastic = [ 1 for _ in range(len(titles))]
my_dataset = pandas.DataFrame({'is_sarcastic': is_sarcastic,
    'titles':titles, 
    'article_link': article_link})

pandas.DataFrame({'titles':titles,
                  'article_link':article_link,
                  'is_sarcastic':is_sarcastic})

,titles,article_link,is_sarcastic
0,Researchers Predict Pandemic Could Result In 5...,https://www.theonion.com/researchers-predict-p...,1
1,Nancy Pelosi Calls Jamaal Bowman To Scold Him ...,https://politics.theonion.com/nancy-pelosi-cal...,1
2,"Gamers, We Just Spent 4 Days Trapped In A Roll...",https://ogn.theonion.com/gamers-we-just-spent-...,1
3,New Ultra-Reinforced Confederate Statue Includ...,https://www.theonion.com/new-ultra-reinforced-...,1
4,Defiant Florida Officials Announce They Will I...,https://www.theonion.com/defiant-florida-offic...,1
...,...,...,...
1499,Clinton Blasts Obama For Slamming Edwards Jab,https://politics.theonion.com/clinton-blasts-o...,1
1500,Bin Laden Called 'Virtually Impotent',https://politics.theonion.com/bin-laden-called...,1
1501,Heartbroken Bush Runs After Departing Rove's Car,https://politics.theonion.com/heartbroken-bush...,1
1502,Senator Craig Arrested,https://politics.theonion.com/senator-craig-ar...,1


####1.4. Export dữ liệu

Sau khi hoàn thành xử lý dữ liệu, ta cần export ra một file .csv riêng.

In [7]:
my_dataset.to_csv("onion_titles.csv")

####1.5. Gộp dữ liệu

Thực hiện tương tự với trang ***HuffPost***:
[The HuffPost Web Crawler](https://colab.research.google.com/drive/1UajgvAIteWdaGKwpjJATa87OxACeJeNE?usp=sharing&fbclid=IwAR2hwLsAYnrtxIQXNjiDAjQXGgE1_CiaT9wGrt9BGcVWKlhLWhe9O_q3Sb8#scrollTo=x29oW97W2Clh)

Tới bước này, ta cần gộp dữ liệu từ hai trang trên thành 1 file .csv duy nhất để phục vụ cho việc test performance của model. 